In [2]:
from matplotlib import pyplot as plt
from PIL import Image
import numpy as np
import torch
import torch.nn as nn

In [13]:
import os
from torch.utils.data import Dataset
import torchvision.transforms as transforms

import os
from PIL import Image
from torch.utils.data import Dataset
import torchvision.transforms as transforms
import torch
import numpy as np

class CustomCityscapesDataset(Dataset):
    def __init__(self, image_dir, label_dir, transform=None, load_color_labels=False):
        self.image_dir = image_dir
        self.label_dir = label_dir
        self.transform = transform
        self.load_color_labels = load_color_labels  # Load both label types
        self.images = []
        self.labels_ids = []
        self.labels_colors = []  # For color visualization

        # List all image and label file pairs
        for folder in os.listdir(image_dir):
            folder_image_dir = os.path.join(image_dir, folder)
            folder_label_dir = os.path.join(label_dir, folder)

            if os.path.isdir(folder_image_dir) and os.path.isdir(folder_label_dir):
                for image_file in os.listdir(folder_image_dir):
                    if image_file.endswith("_leftImg8bit.jpg"):
                        # Corresponding label (ID-based)
                        label_id_file = image_file.replace("_leftImg8bit.jpg", "_gtFine_labellevel3Ids.png")
                        # Corresponding label (color-coded)
                        label_color_file = image_file.replace("_leftImg8bit.jpg", "_gtFine_labelColors.png")

                        # Full paths
                        self.images.append(os.path.join(folder_image_dir, image_file))
                        self.labels_ids.append(os.path.join(folder_label_dir, label_id_file))
                        self.labels_colors.append(os.path.join(folder_label_dir, label_color_file))

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image_path = self.images[idx]
        label_id_path = self.labels_ids[idx]
        label_color_path = self.labels_colors[idx] if self.load_color_labels else None

        # Open image and label
        image = Image.open(image_path).convert("RGB")
        label_id = Image.open(label_id_path)
        label_color = Image.open(label_color_path) if label_color_path else None

        # Apply transforms to the image (e.g., resizing, normalization, etc.)
        if self.transform:
            image = self.transform(image)

        # Convert label ID to tensor where each pixel is treated as a class ID
        label_id = torch.tensor(np.array(label_id), dtype=torch.long)

        # Optional: Load color label if requested
        if label_color:
            label_color = transforms.ToTensor()(label_color)  # Convert color label to a tensor

        return (image, label_id, label_color) if label_color else (image, label_id)

# Example usage:
image_dir = "D:/New folder/Inter_Bootcamp/dataset/train"
label_dir = "D:/New folder/Inter_Bootcamp/dataset/labels"

# Define transforms for the input image
transform = transforms.Compose([
    transforms.Resize((256, 256)),  # Resize as per your needs
    transforms.ToTensor(),  # Convert to tensor and normalize
])

# If you want to load color labels for visualization, set `load_color_labels=True`
dataset = CustomCityscapesDataset(image_dir, label_dir, transform=transform, load_color_labels=True)



In [14]:
from torch.utils.data import DataLoader

batch_size = 8  # Adjust this based on your GPU/CPU capability
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [15]:
import json

class CustomDatasetWithPolygons(CustomDataset):
    def __getitem__(self, idx):
        image, label = super().__getitem__(idx)
        
        # Loading corresponding polygons JSON
        json_path = self.images[idx].replace("_leftImg8bit.jpg", "_gtFine_polygons.json")
        with open(json_path, 'r') as f:
            polygons_data = json.load(f)

        # You can process polygon data here

        return image, label, polygons_data

In [19]:
from torchvision.datasets import Cityscapes
from matplotlib import pyplot as plt
from PIL import Image
import numpy as np
import torch
import torch.nn as nn
import os
import json
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms

# Custom Cityscapes Dataset
class CustomCityscapesDataset(Dataset):
    def __init__(self, image_dir, label_dir, transform=None, load_color_labels=False):
        self.image_dir = image_dir
        self.label_dir = label_dir
        self.transform = transform
        self.load_color_labels = load_color_labels  # Load both label types
        self.images = []
        self.labels_ids = []
        self.labels_colors = []  # For color visualization

        # List all image and label file pairs
        for folder in os.listdir(image_dir):
            folder_image_dir = os.path.join(image_dir, folder)
            folder_label_dir = os.path.join(label_dir, folder)

            if os.path.isdir(folder_image_dir) and os.path.isdir(folder_label_dir):
                for image_file in os.listdir(folder_image_dir):
                    if image_file.endswith("_leftImg8bit.jpg"):
                        # Corresponding label (ID-based)
                        label_id_file = image_file.replace("_leftImg8bit.jpg", "_gtFine_labellevel3Ids.png")
                        # Corresponding label (color-coded)
                        label_color_file = image_file.replace("_leftImg8bit.jpg", "_gtFine_labelColors.png")

                        # Full paths
                        self.images.append(os.path.join(folder_image_dir, image_file))
                        self.labels_ids.append(os.path.join(folder_label_dir, label_id_file))
                        self.labels_colors.append(os.path.join(folder_label_dir, label_color_file))

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image_path = self.images[idx]
        label_id_path = self.labels_ids[idx]
        label_color_path = self.labels_colors[idx] if self.load_color_labels else None

        # Open image and label
        image = Image.open(image_path).convert("RGB")
        label_id = Image.open(label_id_path)
        label_color = Image.open(label_color_path) if label_color_path else None

        # Apply transforms to the image (e.g., resizing, normalization, etc.)
        if self.transform:
            image = self.transform(image)

        # Convert label ID to tensor where each pixel is treated as a class ID
        label_id = torch.tensor(np.array(label_id), dtype=torch.long)

        # Optional: Load color label if requested
        if label_color:
            label_color = transforms.ToTensor()(label_color)  # Convert color label to a tensor

        return (image, label_id, label_color) if label_color else (image, label_id)

# Dataset with polygons
class CustomDatasetWithPolygons(CustomCityscapesDataset):
    def __getitem__(self, idx):
        image, label = super().__getitem__(idx)
        
        # Loading corresponding polygons JSON
        json_path = self.images[idx].replace("_leftImg8bit.jpg", "_gtFine_polygons.json")
        with open(json_path, 'r') as f:
            polygons_data = json.load(f)

        # You can process polygon data here
        return image, label, polygons_data

# Albumentations for data augmentation
import albumentations as A
from albumentations.pytorch import ToTensorV2

transform = A.Compose([
    A.Resize(256, 512),
    A.HorizontalFlip(),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2(),
])

# Utility functions for label encoding/decoding
ignore_index = 255
valid_classes = [ignore_index, 7, 8, 11, 12, 13, 17, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 31, 32, 33]
n_classes = len(valid_classes)

class_map = dict(zip(valid_classes, range(n_classes)))
colors = [
    [0, 0, 0], [128, 64, 128], [244, 35, 232], [70, 70, 70], [102, 102, 156], 
    [190, 153, 153], [153, 153, 153], [250, 170, 30], [220, 220, 0], [107, 142, 35], 
    [152, 251, 152], [0, 130, 180], [220, 20, 60], [255, 0, 0], [0, 0, 142], 
    [0, 0, 70], [0, 60, 100], [0, 80, 100], [0, 0, 230], [119, 11, 32]
]
label_colours = dict(zip(range(n_classes), colors))

def encode_segmap(mask):
    for _validc in valid_classes:
        mask[mask == _validc] = class_map[_validc]
    return mask

def decode_segmap(temp):
    r, g, b = temp.copy(), temp.copy(), temp.copy()
    for l in range(n_classes):
        r[temp == l], g[temp == l], b[temp == l] = label_colours[l]
    rgb = np.stack([r, g, b], axis=2)
    return rgb / 255.0

# Model definition
import segmentation_models_pytorch as smp
from pytorch_lightning import LightningModule, Trainer
import torchmetrics

class OurModel(LightningModule):
    def __init__(self):
        super(OurModel, self).__init__()
        self.model = smp.Unet(
            encoder_name="resnet34", 
            encoder_weights="imagenet", 
            in_channels=3, 
            classes=n_classes
        )
        self.lr = 1e-3
        self.criterion = smp.losses.DiceLoss(mode='multiclass')
        self.metrics = torchmetrics.IoU(num_classes=n_classes)
    
    def forward(self, x):
        return self.model(x)
    
    def configure_optimizers(self):
        return torch.optim.AdamW(self.parameters(), lr=self.lr)
    
    def training_step(self, batch, batch_idx):
        img, seg = batch
        output = self(img)
        seg = encode_segmap(seg)
        loss = self.criterion(output, seg)
        iou = self.metrics(output, seg)
        self.log('train_loss', loss, on_epoch=True)
        self.log('train_iou', iou, on_epoch=True)
        return loss

    def validation_step(self, batch, batch_idx):
        img, seg = batch
        output = self(img)
        seg = encode_segmap(seg)
        loss = self.criterion(output, seg)
        iou = self.metrics(output, seg)
        self.log('val_loss', loss, on_epoch=True)
        self.log('val_iou', iou, on_epoch=True)
        return loss

# Dataloaders
train_dataset = CustomCityscapesDataset(
    image_dir="D:/New folder/Inter_Bootcamp/dataset/train",
    label_dir="D:/New folder/Inter_Bootcamp/dataset/labels",
    transform=transform
)

val_dataset = CustomCityscapesDataset(
    image_dir="D:/New folder/Inter_Bootcamp/dataset/val",
    label_dir="D:/New folder/Inter_Bootcamp/dataset/labels",
    transform=transform
)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

# Training
checkpoint_callback = ModelCheckpoint(monitor='val_loss', save_top_k=1)
trainer = Trainer(
    max_epochs=200, 
    gpus=1 if torch.cuda.is_available() else 0, 
    precision=16, 
    callbacks=[checkpoint_callback]
)
model = OurModel()
trainer.fit(model, train_loader, val_loader)

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'D:/New folder/Inter_Bootcamp/dataset/val'

In [22]:
import os
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from PIL import Image
import numpy as np
from matplotlib import pyplot as plt
import json
import segmentation_models_pytorch as smp
import torchmetrics
from pytorch_lightning import LightningModule, Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from torchmetrics import JaccardIndex

# Custom dataset class
class CustomCityscapesDataset(Dataset):
    def __init__(self, image_dir, label_dir, transform=None, load_color_labels=False):
        self.image_dir = image_dir
        self.label_dir = label_dir
        self.transform = transform
        self.load_color_labels = load_color_labels
        self.images = []
        self.labels_ids = []
        self.labels_colors = []

        for folder in os.listdir(image_dir):
            folder_image_dir = os.path.join(image_dir, folder)
            folder_label_dir = os.path.join(label_dir, folder)

            if os.path.isdir(folder_image_dir) and os.path.isdir(folder_label_dir):
                for image_file in os.listdir(folder_image_dir):
                    if image_file.endswith("_leftImg8bit.jpg"):
                        label_id_file = image_file.replace("_leftImg8bit.jpg", "_gtFine_labellevel3Ids.png")
                        label_color_file = image_file.replace("_leftImg8bit.jpg", "_gtFine_labelColors.png")

                        self.images.append(os.path.join(folder_image_dir, image_file))
                        self.labels_ids.append(os.path.join(folder_label_dir, label_id_file))
                        self.labels_colors.append(os.path.join(folder_label_dir, label_color_file))

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image_path = self.images[idx]
        label_id_path = self.labels_ids[idx]
        label_color_path = self.labels_colors[idx] if self.load_color_labels else None

        image = Image.open(image_path).convert("RGB")
        label_id = Image.open(label_id_path)
        label_color = Image.open(label_color_path) if label_color_path else None

        if self.transform:
            image = self.transform(image)

        label_id = torch.tensor(np.array(label_id), dtype=torch.long)

        if label_color:
            label_color = transforms.ToTensor()(label_color)

        return (image, label_id, label_color) if label_color else (image, label_id)


# Split the train dataset into train/val
def split_train_val(dataset, val_split=0.2):
    val_size = int(len(dataset) * val_split)
    train_size = len(dataset) - val_size
    train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
    return train_dataset, val_dataset


# Model class
class OurModel(LightningModule):
    def __init__(self, train_dataset, val_dataset, n_classes, transform):
        super(OurModel, self).__init__()
        self.layer = smp.Unet(
            encoder_name="resnet34",  # choose encoder
            encoder_weights="imagenet",  # use imagenet pre-trained weights
            in_channels=3,  # model input channels (RGB images)
            classes=n_classes  # model output channels (number of classes in your dataset)
        )
        self.lr = 1e-3
        self.batch_size = 16
        self.criterion = smp.losses.DiceLoss(mode='multiclass')
        self.metrics = torchmetrics.JaccardIndex(task='multiclass',num_classes=n_classes)
        self.train_class = train_dataset
        self.val_class = val_dataset

    def process(self, image, segment):
        out = self(image)
        loss = self.criterion(out, segment.long())
        iou = self.metrics(out, segment)
        return loss, iou

    def forward(self, x):
        return self.layer(x)

    def configure_optimizers(self):
        return torch.optim.AdamW(self.parameters(), lr=self.lr)

    def train_dataloader(self):
        return DataLoader(self.train_class, batch_size=self.batch_size, shuffle=True)

    def training_step(self, batch, batch_idx):
        image, segment = batch
        loss, iou = self.process(image, segment)
        self.log('train_loss', loss, on_epoch=True)
        self.log('train_iou', iou, on_epoch=True)
        return loss

    def val_dataloader(self):
        return DataLoader(self.val_class, batch_size=self.batch_size, shuffle=False)

    def validation_step(self, batch, batch_idx):
        image, segment = batch
        loss, iou = self.process(image, segment)
        self.log('val_loss', loss, on_epoch=True)
        self.log('val_iou', iou, on_epoch=True)
        return loss


# Training the model
image_dir = "D:/New folder/Inter_Bootcamp/dataset/train"
label_dir = "D:/New folder/Inter_Bootcamp/dataset/labels"

# Define transforms
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()
])

# Create the dataset
dataset = CustomCityscapesDataset(image_dir, label_dir, transform=transform)

# Split into train and val
train_dataset, val_dataset = split_train_val(dataset)

n_classes = 40  # Define your number of classes based on your dataset

# Initialize model
model = OurModel(train_dataset, val_dataset, n_classes, transform)

# Checkpoints
checkpoint_callback = ModelCheckpoint(monitor='val_loss', dirpath='checkpoints', filename='model', save_last=True)

# Training
trainer = Trainer(max_epochs=50, accelerator="auto", callbacks=[checkpoint_callback])
trainer.fit(model)


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
C:\Users\prakh\anaconda3\Lib\site-packages\pytorch_lightning\trainer\connectors\logger_connector\logger_connector.py:75: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default

  | Name      | Type                   | Params | Mode 
-------------------------------------------------------------
0 | layer     | Unet                   | 24.4 M | train
1 | criterion | DiceLoss               | 0      | train
2 | metrics   | MulticlassJaccardIndex | 0      | train
-------------------------------------------------------------
24.4 M    Tra

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

C:\Users\prakh\anaconda3\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


RuntimeError: Class values must be smaller than num_classes.